# Asset Portfolio Management using Deep Reinforcement Learning
---

## 3.0 Feature Engineering and Data Preprocessing
---
We perform feature engineering and data preprocessing by:
* Adding Technical Indicators to the data. The technical inicators are used as inputs in the training of our Reinforcement Learning Model
* Adding Coveriance Matrices which is also used as input for training the Models
* Splitting the data into the training set and the testing (trading) set

### 3.1 Import Relevant Libraries

In [1]:
import pandas as pd
import numpy as np
import ta
from ta import add_all_ta_features
from ta.utils import dropna
from finrl.preprocessing.data import data_split

### 3.2 Load the Data from the csv Files

In [2]:
# Load the whole data set
data = pd.read_csv('./datasets/data.csv')

# Load the close prices dataset
prices_data = pd.read_csv('./datasets/close_prices.csv')

### 3.3 Add Technical Indicators
---
We define a function to add technical indicators to the dataset by making use of the ta library

The folloing indicators are considered:
* Volatility Average True Range (ATR)
* Volatility Bollinger Band Width (BBW)
* Volume On-balance Volume (OBV
* Volume Chaikin Money Flow (CMF)
* Trend Moving Average Convergence Divergence (MACD)
* Trend Average Directional Index (ADX)
* Trend Fast Simple Moving Average (SMA)
* Trend Fast Exponential Moving Average (EMA)
* Trend Commodity Channel Index (CCI)
* Momentum Relative Strength Index (RSI)

In [3]:
# Define a Function for adding technical indicators

def add_features(data, feature_list, short_names):
    """
    Function to add technical indicators for features
    -Takes in a dataset with Open, High, Low, Close and Volume
    -Also takes in a list of the technical indicators to be added 
     as well as a list of the shortened indicator names
    """
    
    # list of column names to filter the features
    data_col_names = list(data.columns)
    filter_names = data_col_names + feature_list
    col_rename = data_col_names +  short_names
    
    # Add technical indicators using the ta Library
    data = add_all_ta_features(data, open="open", high="high", 
    low="low", close="close", volume="volume") 
    
    # Filter the Indicators with the required features
    data = data[filter_names]
    data.columns = col_rename # rename the columns to use shortened indicator names
    data = data.dropna()
    
    return data

In [4]:
# List of Features to add
feature_list= ['volatility_atr','volatility_bbw','volume_obv','volume_cmf',
               'trend_macd', 'trend_adx', 'trend_sma_fast', 
               'trend_ema_fast', 'trend_cci', 'momentum_rsi']

# Short names of the features
short_names = ['atr', 'bbw','obv','cmf','macd', 'adx', 'sma', 'ema', 'cci', 'rsi']

#feature_list= ['volatility_atr','volatility_bbw','volume_obv','volume_cmf','trend_macd']

# Short names of the features
#short_names = ['atr', 'bbw','obv','cmf','macd']

In [5]:
# Add Indicators to our dataset
data_with_features = data.copy()

data_with_features = add_features(data_with_features, feature_list, short_names)

C:\Users\Musonda\anaconda3\lib\site-packages\ta\trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
C:\Users\Musonda\anaconda3\lib\site-packages\ta\trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])


In [6]:
data_with_features.head()

,date,open,high,low,close,volume,tic,atr,bbw,obv,cmf,macd,adx,sma,ema,cci,rsi
25,2009-02-06,3.465000,3.571429,3.464286,3.072270,687209600.0,AAPL,0.522545,22.543810,1.974986e+09,-7.447842,0.075614,0.000000,2.846337,2.858466,163.772057,69.370215
26,2009-02-09,3.571429,3.678571,3.553571,3.158227,715010800.0,AAPL,0.530921,25.755168,2.689996e+09,-7.411879,0.093926,0.000000,2.882666,2.904583,180.133487,72.498457
27,2009-02-10,3.618929,3.661072,3.466429,3.014041,849060800.0,AAPL,0.528113,26.666193,1.840936e+09,-7.324339,0.095700,22.201686,2.906980,2.921423,124.407157,61.206266
28,2009-02-11,3.441786,3.511071,3.420357,2.982924,674973600.0,AAPL,0.525005,27.059048,1.165962e+09,-7.455972,0.093517,22.287457,2.925414,2.930884,84.413093,59.067994
29,2009-02-12,3.422500,3.562500,3.422500,3.058405,817188400.0,AAPL,0.530462,27.371890,1.983150e+09,-7.255083,0.096763,22.653364,2.947339,2.950503,91.828157,62.491148


### 3.4 Add Covariance Matrix
---
We define a function that will add Covarance Matrices to our dataset

In [7]:
def add_cov_matrix(df):
    """
    Function to add Coveriance Matrices as part of the defined states
    """
    # Sort the data and index by date and tic
    df=df.sort_values(['date','tic'],ignore_index=True) 
    df.index = df.date.factorize()[0]
    
    cov_list = [] # create empty list for storing coveriance matrices at each time step
    
    # look back for constructing the coveriance matrix is one year
    lookback=252
    for i in range(lookback,len(df.index.unique())):
        data_lookback = df.loc[i-lookback:i,:]
        price_lookback=data_lookback.pivot_table(index = 'date',columns = 'tic', values = 'close')
        return_lookback = price_lookback.pct_change().dropna()
        covs = return_lookback.cov().values 
        covs = covs/covs.max()
        cov_list.append(covs)
        
    df_cov = pd.DataFrame({'date':df.date.unique()[lookback:],'cov_list':cov_list})
    df = df.merge(df_cov, on='date')
    df = df.sort_values(['date','tic']).reset_index(drop=True)
    
    return df

In [8]:
# Add Covariance Matrices to our dataset
data_with_features_covs = data_with_features.copy()
data_with_features_covs = add_cov_matrix(data_with_features_covs)

In [9]:
data_with_features_covs.head()

,date,open,high,low,close,volume,tic,atr,bbw,obv,cmf,macd,adx,sma,ema,cci,rsi,cov_list
0,2009-12-31,7.611786,7.619643,7.520000,6.492372,352410800.0,AAPL,1.089849,14.391304,1.847973e+10,-14.498982,0.106168,20.032052,6.237017,6.309319,141.146843,64.127926,"[[0.17945278689647873, 0.22869766853329948, 0...."
1,2009-12-31,40.900002,41.080002,40.490002,34.291534,4030500.0,AXP,6.638453,7.839731,4.883513e+10,-14.537243,0.324763,23.340487,34.681527,34.582270,16.078828,50.694489,"[[0.17945278689647873, 0.22869766853329948, 0...."
2,2009-12-31,55.000000,55.220001,54.049999,41.856789,2189400.0,BA,12.891344,4.916607,6.798863e+10,-26.245485,0.444625,16.902318,42.400007,42.370060,-75.723504,49.726875,"[[0.17945278689647873, 0.22869766853329948, 0...."
3,2009-12-31,57.599998,57.959999,56.990002,41.246521,3859700.0,CAT,16.411522,4.599234,6.202233e+10,-35.806296,0.044336,10.020770,41.918410,41.881607,-50.733730,44.302408,"[[0.17945278689647873, 0.22869766853329948, 0...."
4,2009-12-31,24.100000,24.170000,23.940001,17.903095,25208100.0,CSCO,6.172343,4.656155,6.810164e+10,-36.494340,0.060804,8.984294,17.752903,17.857885,57.386845,53.264455,"[[0.17945278689647873, 0.22869766853329948, 0...."


### 3.5 Split the Data
---
We will split both the close prices and the whole dataset into train and test (trade) data.

We will use 80% of the data for training and then test on the remaining 20%.

We will make use of the FinRL Library function of data_split to split our data into train and test

In [10]:
# Define the start and end dates for the train and test data

train_pct = 0.8 # percentage of train data
date_list = list(data_with_features_covs.date.unique()) # List of dates in the data
date_list_len = len(date_list) # len of the date list
train_data_len = int(train_pct * date_list_len) # length of the train data

train_start_date = date_list[0]
train_end_date = date_list[train_data_len]

test_start_date = date_list[train_data_len+1]
test_end_date = date_list[-1]

In [11]:
# Split the whole dataset
train_data = data_split(data_with_features_covs, train_start_date, train_end_date)
test_data = data_split(data_with_features_covs, test_start_date, test_end_date)

# Split the Close Prices dataset
prices_train_data = prices_data[prices_data['date']<=train_end_date]
prices_test_data = prices_data[prices_data['date']>=test_start_date]

### 3.6 Store the Dataframes

In [12]:
prices_train = prices_train_data.copy()
prices_test = prices_test_data.copy()

train_df = train_data.copy()
test_df = test_data.copy()

In [13]:
train_df.head()

,date,open,high,low,close,volume,tic,atr,bbw,obv,cmf,macd,adx,sma,ema,cci,rsi,cov_list
0,2009-12-31,7.611786,7.619643,7.520000,6.492372,352410800.0,AAPL,1.089849,14.391304,1.847973e+10,-14.498982,0.106168,20.032052,6.237017,6.309319,141.146843,64.127926,"[[0.17945278689647873, 0.22869766853329948, 0...."
0,2009-12-31,40.900002,41.080002,40.490002,34.291534,4030500.0,AXP,6.638453,7.839731,4.883513e+10,-14.537243,0.324763,23.340487,34.681527,34.582270,16.078828,50.694489,"[[0.17945278689647873, 0.22869766853329948, 0...."
0,2009-12-31,55.000000,55.220001,54.049999,41.856789,2189400.0,BA,12.891344,4.916607,6.798863e+10,-26.245485,0.444625,16.902318,42.400007,42.370060,-75.723504,49.726875,"[[0.17945278689647873, 0.22869766853329948, 0...."
0,2009-12-31,57.599998,57.959999,56.990002,41.246521,3859700.0,CAT,16.411522,4.599234,6.202233e+10,-35.806296,0.044336,10.020770,41.918410,41.881607,-50.733730,44.302408,"[[0.17945278689647873, 0.22869766853329948, 0...."
0,2009-12-31,24.100000,24.170000,23.940001,17.903095,25208100.0,CSCO,6.172343,4.656155,6.810164e+10,-36.494340,0.060804,8.984294,17.752903,17.857885,57.386845,53.264455,"[[0.17945278689647873, 0.22869766853329948, 0...."


In [14]:
%store prices_train
%store prices_test

%store train_df
%store test_df

Stored 'prices_train' (DataFrame)
Stored 'prices_test' (DataFrame)
Stored 'train_df' (DataFrame)
Stored 'test_df' (DataFrame)
